# Intent Classification

## Shallow Machine Learning

In [23]:
pip install nltk scikit-learn pandas

In [24]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.metrics import classification_report

In [25]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### NLP Tools Initialization

In [26]:
# create stemmer
stemmer = PorterStemmer()

# Download NLTK data for tokenization and stopword removal
nltk.download('punkt')
nltk.download('stopwords')

# Load Indonesian stopwords from NLTK
stop_words = set(stopwords.words('english'))

# Initialize the CountVectorizer (Bag of Words model)
binary_vectorizer = CountVectorizer(binary=True) # binary as value
tf_vectorizer = CountVectorizer() # count as value
tfidf_vectorizer = TfidfVectorizer() # tfidf as value

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Preprocessing
1. lowercase<br>
2. tokenization<br>
3. removing puctuation<br>
4. stemming<br>

In [27]:
# Define a function to preprocess the text
def preprocess_text(text):
  # 1. Convert text to lowercase
  text = text.lower()

  # 2. Tokenize the text
  tokens = word_tokenize(text)

  # 3. Remove punctuation
  tokens = [word for word in tokens if word not in string.punctuation]

  # 4. Stem the tokens
  tokens = [stemmer.stem(word) for word in tokens]

  # Rejoin the tokens into a single string
  processed_text = ' '.join(tokens)

  return processed_text

### Training and Evaluation Pipeline

In [28]:
# Define a function to display the metrics
def metrics(y_test, y_pred):
  # Calculate metrics
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='macro')
  recall = recall_score(y_test, y_pred, average='macro')
  f1_score_value = f1_score(y_test, y_pred, average='macro')

  print(f'Accuracy  : {accuracy:.4f}')
  print(f'Precision : {precision:.4f}')
  print(f'Recall    : {recall:.4f}')
  print(f'F1-score  : {f1_score_value:.4f}')

  # Generate and display classification report
  report = classification_report(y_test, y_pred)
  print(report)

  # Generate confusion matrix
  confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
  print("Confusion Matrix")
  print(confusion_matrix)



def train_and_evaluate(classifier, X_train_binary, X_train_tf, X_train_tfidf, y_train, X_test_binary, X_test_tf, X_test_tfidf, y_test):
  # Train and make predictions
  ## Binary
  classifier.fit(X_train_binary, y_train)
  y_pred_binary = classifier.predict(X_test_binary)

  print("========================= Binary =========================")
  metrics(y_test, y_pred_binary)

  ## TF
  classifier.fit(X_train_tf, y_train)
  y_pred_tf = classifier.predict(X_test_tf)

  print("========================= TF =========================")
  metrics(y_test, y_pred_tf)

  ## TFxIDF
  classifier.fit(X_train_tfidf, y_train)
  y_pred_tfidf = classifier.predict(X_test_tfidf)

  print("========================= TFxIDF =========================")
  metrics(y_test, y_pred_tfidf)

### Split Data

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive

# # Step 1: Read the Excel file
# drive.mount('/content/drive')

# file_path = '/content/drive/My Drive/SEMESTER 7/IF5153 Natural Language Processing/preprocessed.xlsx'
# sheets = ['Interrogative Final', 'Imperative Final', 'Declarative Final']

# # Combine all sheets
# dataframes = [pd.read_excel(file_path, sheet_name=sheet) for sheet in sheets]
# data = pd.concat(dataframes, ignore_index=True)

# # Optional: Add a column to label the sheet each row came from (useful for debugging or tracking)
# # for i, sheet in enumerate(sheets):
# #     dataframes[i]['label'] = sheet

# # Step 2: Shuffle the data
# shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# # Step 3: Split into training and testing sets (80:20)
# train_data, test_data = train_test_split(shuffled_data, test_size=0.2, random_state=42)

# # Step 4: Save to new files (optional)
# train_data.to_excel('/content/drive/My Drive/SEMESTER 7/IF5153 Natural Language Processing/train_data.xlsx', index=False)
# test_data.to_excel('/content/drive/My Drive/SEMESTER 7/IF5153 Natural Language Processing/test_data.xlsx', index=False)

# print("Data split completed.")


### Preprocessed Data

In [30]:
drive.mount('/content/drive')

train_file_path = '/content/drive/My Drive/SEMESTER 7/IF5153 Natural Language Processing/train_data.xlsx'
test_file_path = '/content/drive/My Drive/SEMESTER 7/IF5153 Natural Language Processing/test_data.xlsx'

train_data = pd.read_excel(train_file_path)
test_data = pd.read_excel(test_file_path)


# Preprocess the train data
train_data['preprocessed'] = train_data['Sentence'].apply(preprocess_text)

# Preprocess the test data
test_data['preprocessed'] = test_data['Sentence'].apply(preprocess_text)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Bag of Words using Binary, TF, TFxIDF

In [31]:
# Example preprocessed text data
train_preprocessed_texts = train_data['preprocessed']  # This column contains the preprocessed text
test_preprocessed_texts = test_data['preprocessed']

y_train = train_data['Intent']
y_test = test_data['Intent']

# Fit and transform the preprocessed text data to generate the Bag of Words matrix
## binary
X_train_binary = binary_vectorizer.fit_transform(train_preprocessed_texts)
X_test_binary = binary_vectorizer.transform(test_preprocessed_texts)

## TF
X_train_tf = tf_vectorizer.fit_transform(train_preprocessed_texts)
X_test_tf = tf_vectorizer.transform(test_preprocessed_texts)

## TFxIDF
X_train_tfidf = tfidf_vectorizer.fit_transform(train_preprocessed_texts)
X_test_tfidf = tfidf_vectorizer.transform(test_preprocessed_texts)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression classifier
classifier = LogisticRegression(max_iter=500)

# Train and Evaluate
train_and_evaluate(classifier, X_train_binary, X_train_tf, X_train_tfidf, y_train, X_test_binary, X_test_tf, X_test_tfidf, y_test)

========================= Binary =========================
Accuracy  : 0.8693
Precision : 0.8691
Recall    : 0.8693
F1-score  : 0.8692
               precision    recall  f1-score   support

  Declarative       0.85      0.86      0.85     11545
   Imperative       0.90      0.90      0.90     11686
Interrogative       0.86      0.85      0.85     11788

     accuracy                           0.87     35019
    macro avg       0.87      0.87      0.87     35019
 weighted avg       0.87      0.87      0.87     35019

Confusion Matrix
Predicted      Declarative  Imperative  Interrogative    All
Actual                                                      
Declarative           9877         580           1088  11545
Imperative             558       10574            554  11686
Interrogative         1140         657           9991  11788
All                  11575       11811          11633  35019
========================= TF =========================
Accuracy  : 0.8680
Precision : 0.8678
R

### Support Vector Machines (SVM)

In [ ]:
from sklearn.svm import SVC

# Initialize the SVM classifier
classifier = SVC(kernel='linear')

# Train and Evaluate
train_and_evaluate(classifier, X_train_binary, X_train_tf, X_train_tfidf, y_train, X_test_binary, X_test_tf, X_test_tfidf, y_test)

### k-Nearest Neighbors (k-NN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize the k-NN classifier
classifier = KNeighborsClassifier(n_neighbors=13)

# Train and Evaluate
train_and_evaluate(classifier, X_train_binary, X_train_tf, X_train_tfidf, y_train, X_test_binary, X_test_tf, X_test_tfidf, y_test)

========================= Binary =========================
Accuracy  : 0.6768
Precision : 0.7062
Recall    : 0.6751
F1-score  : 0.6612
               precision    recall  f1-score   support

  Declarative       0.77      0.38      0.51     11545
   Imperative       0.78      0.80      0.79     11686
Interrogative       0.58      0.84      0.68     11788

     accuracy                           0.68     35019
    macro avg       0.71      0.68      0.66     35019
 weighted avg       0.71      0.68      0.66     35019

Confusion Matrix
Predicted      Declarative  Imperative  Interrogative    All
Actual                                                      
Declarative           4416        1756           5373  11545
Imperative             406        9388           1892  11686
Interrogative          948         944           9896  11788
All                   5770       12088          17161  35019
========================= TF =========================
Accuracy  : 0.6713
Precision : 0.6946
R

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Initialize the Naive Bayes classifier
classifier = MultinomialNB()

# Train and Evaluate
train_and_evaluate(classifier, X_train_binary, X_train_tf, X_train_tfidf, y_train, X_test_binary, X_test_tf, X_test_tfidf, y_test)

========================= Binary =========================
Accuracy  : 0.8115
Precision : 0.8150
Recall    : 0.8114
F1-score  : 0.8122
               precision    recall  f1-score   support

  Declarative       0.79      0.80      0.79     11545
   Imperative       0.89      0.81      0.85     11686
Interrogative       0.77      0.83      0.80     11788

     accuracy                           0.81     35019
    macro avg       0.82      0.81      0.81     35019
 weighted avg       0.82      0.81      0.81     35019

Confusion Matrix
Predicted      Declarative  Imperative  Interrogative    All
Actual                                                      
Declarative           9216         432           1897  11545
Imperative            1236        9446           1004  11686
Interrogative         1280         751           9757  11788
All                  11732       10629          12658  35019
========================= TF =========================
Accuracy  : 0.8100
Precision : 0.8133
R

## Deep Learning

In [32]:
# Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, SimpleRNN, LSTM, Dense, Dropout

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000)  # same vocab size as during training
tokenizer.fit_on_texts(train_data['Sentence'])

X_train_DL = tokenizer.texts_to_sequences(train_data['Sentence'])
X_test_DL = tokenizer.texts_to_sequences(test_data['Sentence'])

X_train_DL = pad_sequences(X_train_DL, maxlen=100)
X_test_DL = pad_sequences(X_test_DL, maxlen=100)

y_train = train_data['Intent']
y_test = test_data['Intent']

In [35]:
import json
tokenizer_json = tokenizer.to_json()
with open('cnn_tokenizer.json', 'w') as json_file:
    json.dump(tokenizer_json, json_file)

In [17]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_onehot = to_categorical(y_train_encoded)
y_test_onehot = to_categorical(y_test_encoded)

In [18]:
print(X_test_DL[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    9    7    1  103 1339   10
  974  402]


In [19]:
# Example usage:
input_dim = 10000  # Size of vocabulary
input_length = 100  # Length of input sequences (after padding)
num_classes = 3

### CNN

In [20]:
# Define the CNN model
def create_cnn_model(input_dim, input_length, num_classes):
    model = Sequential()

    # Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=100, input_length=input_length))

    # Convolutional layer
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    # Flatten layer
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

cnn_model = create_cnn_model(input_dim, input_length, num_classes)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
cnn_model.fit(X_train_DL, y_train_onehot, validation_data=(X_test_DL, y_test_onehot), epochs=2, batch_size=32)

Epoch 1/2
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 273s 61ms/step - accuracy: 0.8578 - loss: 0.3647 - val_accuracy: 0.9452 - val_loss: 0.1716
Epoch 2/2
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 298s 55ms/step - accuracy: 0.9581 - loss: 0.1407 - val_accuracy: 0.9457 - val_loss: 0.1779


In [22]:
cnn_model.save("cnn_model.keras")

### RNN

In [ ]:
# Define the RNN model
def create_rnn_model(input_dim, input_length, num_classes):
    model = Sequential()

    # Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=100, input_length=input_length))

    # RNN layer
    model.add(SimpleRNN(128, activation='relu'))

    # Fully connected layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

rnn_model = create_rnn_model(input_dim, input_length, num_classes)
rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
rnn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### LSTM

In [ ]:
# Define the LSTM model
def create_lstm_model(input_dim, input_length, num_classes):
    model = Sequential()

    # Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=100, input_length=input_length))

    # LSTM layer
    model.add(LSTM(128, activation='relu', return_sequences=False))

    # Fully connected layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    return model
lstm_model = create_lstm_model(input_dim, input_length, num_classes)
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

#### Train

In [ ]:
cnn_model.fit(X_train_DL, y_train_onehot, validation_data=(X_test_DL, y_test_onehot), epochs=10, batch_size=32)

Epoch 1/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 234s 53ms/step - accuracy: 0.8673 - loss: 0.3421 - val_accuracy: 0.9440 - val_loss: 0.1807
Epoch 2/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 226s 52ms/step - accuracy: 0.9586 - loss: 0.1388 - val_accuracy: 0.9439 - val_loss: 0.1845
Epoch 3/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 238s 54ms/step - accuracy: 0.9726 - loss: 0.0928 - val_accuracy: 0.9424 - val_loss: 0.2093
Epoch 4/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 243s 50ms/step - accuracy: 0.9803 - loss: 0.0653 - val_accuracy: 0.9362 - val_loss: 0.2599
Epoch 5/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 215s 49ms/step - accuracy: 0.9857 - loss: 0.0466 - val_accuracy: 0.9366 - val_loss: 0.3248
Epoch 6/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 274s 52ms/step - accuracy: 0.9898 - loss: 0.0332 - val_accuracy: 0.9296 - val_loss: 0.3974
Epoch 7/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 269s 54ms/step - accuracy: 0.9916 - loss: 0.0275 - val_accuracy: 0.9254 - val_loss: 0.4599
Epoch 8/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 254s 52ms/step - accuracy: 

In [ ]:
rnn_model.fit(X_train_DL, y_train_onehot, validation_data=(X_test_DL, y_test_onehot), epochs=10, batch_size=32)

Epoch 1/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 299s 68ms/step - accuracy: 0.8383 - loss: 0.3952 - val_accuracy: 0.9433 - val_loss: 0.1861
Epoch 2/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 290s 60ms/step - accuracy: 0.9564 - loss: 0.1508 - val_accuracy: 0.9462 - val_loss: 0.1813
Epoch 3/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 318s 59ms/step - accuracy: 0.9680 - loss: 0.1108 - val_accuracy: 0.9422 - val_loss: 0.1963
Epoch 4/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 259s 59ms/step - accuracy: 0.9744 - loss: 0.0873 - val_accuracy: 0.9394 - val_loss: 0.2257
Epoch 5/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 275s 62ms/step - accuracy: 0.9788 - loss: 0.0701 - val_accuracy: 0.9393 - val_loss: 0.2694
Epoch 6/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 279s 64ms/step - accuracy: 0.9830 - loss: 0.0565 - val_accuracy: 0.9393 - val_loss: 0.2945
Epoch 7/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 314s 62ms/step - accuracy: 0.9848 - loss: 0.0478 - val_accuracy: 0.9341 - val_loss: 0.2871
Epoch 8/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 318s 61ms/step - accuracy: 

In [ ]:
lstm_model.fit(X_train_DL, y_train_onehot, validation_data=(X_test_DL, y_test_onehot), epochs=10, batch_size=32)

Epoch 1/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 907s 207ms/step - accuracy: 0.6607 - loss: 363131731968.0000 - val_accuracy: 0.3368 - val_loss: 1.0987
Epoch 2/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 911s 204ms/step - accuracy: 0.3389 - loss: 1.0986 - val_accuracy: 0.3368 - val_loss: 1.0985
Epoch 3/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 945s 209ms/step - accuracy: 0.3358 - loss: 1.0985 - val_accuracy: 0.3297 - val_loss: 1.0986
Epoch 4/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 892s 203ms/step - accuracy: 0.3328 - loss: 1.0985 - val_accuracy: 0.3297 - val_loss: 1.0986
Epoch 5/10
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 889s 195ms/step - accuracy: 0.3316 - loss: 15.4944 - val_accuracy: 0.3368 - val_loss: 1.0984
Epoch 6/10
1227/4378 ━━━━━━━━━━━━━━━━━━━━ 9:22 178ms/step - accuracy: 0.3362 - loss: 1.0985

## LLM


### Qwen

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import torch

model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-1.5B-Instruct')
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-1.5B-Instruct')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [11]:
X_sample_test = test_data['Sentence'][:500]
y_sample_test = test_data['Intent'][:500]

In [12]:
y_sample_pred_Qwen =[]
for sentence in tqdm(X_sample_test, desc="Processing sentences"):
    text = f"""Given the sentence delimited by triple backticks ```{sentence}```, determine the intent into "Interrogative", "Declarative", or "Imperative"
    Answer: """

    inputs = tokenizer(text, return_tensors="pt").to(device)

    outputs = model.generate(**inputs, max_new_tokens=3, temperature=0.1)
    intent = tokenizer.decode(outputs[0], skip_special_tokens=True)
    last_word = intent.split()[-1]  # Split the string and get the last word
    print(last_word)
    y_sample_pred_Qwen.append(last_word)


print(y_sample_pred_Qwen)

Processing sentences:   0%|          | 1/500 [00:05<43:30,  5.23s/it]

Interrogative


Processing sentences:   0%|          | 2/500 [00:09<38:29,  4.64s/it]

Interrogative


Processing sentences:   1%|          | 3/500 [00:15<42:25,  5.12s/it]

...

Interrogative


Processing sentences: 100%|█████████▉| 499/500 [40:41<00:04,  4.74s/it]

Imperative


Processing sentences: 100%|██████████| 500/500 [40:46<00:00,  4.89s/it]

Declarative
['Interrogative', 'Interrogative', 'a', 'Declarative', 'Interrogative', 'Interrogative', '450', 'Declarative', 'Declarative', 'Interrogative', 'Declarative', 'Declarative', 'Imperative', 'Interrogative', 'Declarative', 'Interrogative', 'Imperative', 'Interrogative', 'Interrogative', 'Interrogative', 'Interrogative', 'Declarative', 'Declarative', 'Interrogative', '**', '100', '10,', 'Imperative', 'Imperative', 'Declarative', 'Go', 'Interrogative', 'Declarative', 'Declarative', 'Declarative', 'Declarative', 'Declarative', 'Declarative', 'Interrogative', 'Interrogative', 'Declarative', 'Imperative', 'Declarative', 'Interrogative', 'Interrogative', 'Interrogative', 'is', 'Declarative', 'Interrogative', 'Imperative', 'it', 'Interrogative', 'Interrogative', 'Interrogative', 'Imperative', 'Declarative', 'Declarative', 'Interrogative', 'Interrogative', 'Interrogative', 'Declarative', 'Declarative', 'Declarative', 'Interrogative', 'Interrogative', 'Declarative', '1/2', 'Interrogativ

In [13]:
print("Gemma Performance Measure:")
metrics(y_sample_test, y_sample_pred_Qwen)

Gemma Performance Measure:
Accuracy  : 0.6320
Precision : 0.0652
Recall    : 0.0579
F1-score  : 0.0563
               precision    recall  f1-score   support

           **       0.00      0.00      0.00         0
          1/2       0.00      0.00      0.00         0
          10,       0.00      0.00      0.00         0
          100       0.00      0.00      0.00         0
          2.5       0.00      0.00      0.00         0
          450       0.00      0.00      0.00         0
          50,       0.00      0.00      0.00         0
            A       0.00      0.00      0.00         0
    Athletics       0.00      0.00      0.00         0
  Declarative       0.89      0.86      0.87       170
   Electrical       0.00      0.00      0.00         0
           Go       0.00      0.00      0.00         0
            I       0.00      0.00      0.00         0
   Imperative       0.72      0.21      0.33       171
Interrogative       0.54      0.84      0.66       159
         Open   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

#### Gemma & LLama from Groq API

In [ ]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.6 MB/s eta 0:00:00


In [ ]:
from groq import Groq

api_key = "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"

groq_client = Groq(api_key=api_key)

y_sample_pred_llama = []
y_sample_pred_gemma = []
for sentence in tqdm(X_sample_test, desc="Processing sentences"):
    prompt = f"""Given the sentence delimited by triple backticks ```{sentence}```, determine the intent into "Interrogative", "Declarative", or "Imperative"
    Give the answer directly.
    Answer: """

    completion = groq_client.chat.completions.create(
          model="llama-3.1-70b-versatile",
          messages=[
              {
                  "role": "user",
                  "content": [
                      {
                          "type": "text",
                          "text": prompt
                      }
                  ],
              }
          ],
          temperature=1,
          max_tokens=3,
          top_p=1,
          stream=False,
          stop=None,
    )

    intent = completion.choices[0].message.content
    last_word = intent.split()[-1]  # Split the string and get the last word
    print("Llama:", last_word)
    y_sample_pred_llama.append(last_word)


    completion = groq_client.chat.completions.create(
          model="gemma2-9b-it",
          messages=[
              {
                  "role": "user",
                  "content": [
                      {
                          "type": "text",
                          "text": prompt
                      }
                  ],
              }
          ],
          temperature=1,
          max_tokens=3,
          top_p=1,
          stream=False,
          stop=None,
      )

    intent = completion.choices[0].message.content
    last_word = intent.split()[-1]  # Split the string and get the last word
    print("Gemma:",  last_word)
    y_sample_pred_gemma.append(last_word)

Processing sentences:   0%|          | 1/500 [00:00<01:53,  4.41it/s]

Llama: Imperative
Gemma: Imperative


Processing sentences:   0%|          | 2/500 [00:00<01:54,  4.35it/s]

Llama: Imperative
Gemma: Interrogative


Processing sentences:   1%|          | 3/500 [00:00<01:55,  4.32it/s]

...

Gemma: Interrogative
Llama: Imperative


Processing sentences: 100%|█████████▉| 499/500 [18:32<00:02,  2.22s/it]

Gemma: Imperative
Llama: Declarative


Processing sentences: 100%|██████████| 500/500 [18:34<00:00,  2.23s/it]

Gemma: Declarative


In [ ]:
label_mapping = {
    "imper": "Imperative",
    '"imper': "Imperative",
    "imperative": "Imperative",
    "declar": "Declarative",
    "declarative": "Declarative",
    "inter": "Interrogative",
    '"inter': "Interrogative",
    "interrogative": "Interrogative",
    '"interrog': "Interrogative",
}

y_sample_pred_gemma_cleaned = [label_mapping.get(label.rstrip('?".!').lower(), "Imperative") for label in y_sample_pred_gemma]
y_sample_pred_llama_cleaned = [label_mapping.get(label.rstrip('"?".!').lower(), "Imperative") for label in y_sample_pred_llama]


In [ ]:
print("Gemma Performance Measure:")
metrics(y_sample_test, y_sample_pred_gemma_cleaned)

Gemma Performance Measure:
Accuracy  : 0.8140
Precision : 0.8437
Recall    : 0.8180
F1-score  : 0.8036
               precision    recall  f1-score   support

  Declarative       0.94      0.96      0.95       170
   Imperative       0.92      0.52      0.66       171
Interrogative       0.68      0.97      0.80       159

     accuracy                           0.81       500
    macro avg       0.84      0.82      0.80       500
 weighted avg       0.85      0.81      0.80       500

Confusion Matrix
Predicted      Declarative  Imperative  Interrogative  All
Actual                                                    
Declarative            163           5              2  170
Imperative              10          89             72  171
Interrogative            1           3            155  159
All                    174          97            229  500


In [ ]:
print("Llama Performance Measure:")
metrics(y_sample_test, y_sample_pred_llama_cleaned)

Llama Performance Measure:
Accuracy  : 0.8200
Precision : 0.8295
Recall    : 0.8225
F1-score  : 0.8154
               precision    recall  f1-score   support

  Declarative       0.94      0.94      0.94       170
   Imperative       0.84      0.61      0.71       171
Interrogative       0.71      0.92      0.80       159

     accuracy                           0.82       500
    macro avg       0.83      0.82      0.82       500
 weighted avg       0.83      0.82      0.82       500

Confusion Matrix
Predicted      Declarative  Imperative  Interrogative  All
Actual                                                    
Declarative            160           9              1  170
Imperative               8         104             59  171
Interrogative            2          11            146  159
All                    170         124            206  500
